# Working solution

In [77]:
KNN_main = adata.obsp[adata.uns[model_params['knn_key']]['connectivities_key']].copy()
knn = KNN_main
KNN_tmp = knn[adata.obs.index.isin(adata_samp.obs.index)]
KNN_tmp.data = np.where(KNN_tmp.data > 0, 1, 0)

indices = np.where(KNN_tmp.sum(axis=0) <= model_params['alpha'])[1]
len(indices)

8443

In [78]:
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np

def compile_and_apply_updates(KNN_main, updates_dict):
    """
    Compile updates from updates_dict and apply them to the KNN_main CSR matrix.
    """
    row_updates = []
    col_updates = []
    data_updates = []

    # Compile the latest updates for each unique index
    compiled_updates = {}
    for epoch, (indices, KNN_hop) in sorted(updates_dict.items(), reverse=True):
        for idx, original_idx in enumerate(indices):
            if original_idx not in compiled_updates:
                compiled_updates[original_idx] = KNN_hop[idx, :].tocoo()

    # Accumulate updates
    for original_idx, update_coo in compiled_updates.items():
        row_updates.extend([original_idx] * len(update_coo.data))
        col_updates.extend(update_coo.col)
        data_updates.extend(update_coo.data)

    # Create a COO matrix from the compiled updates
    updates_coo = coo_matrix((data_updates, (row_updates, col_updates)), shape=KNN_main.shape)

    # Convert COO to CSR and apply updates
    KNN_main += updates_coo.tocsr()

    return KNN_main

In [3]:
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
from multiprocessing import Pool, Manager
from functools import partial

def process_update_chunk(shared_row_updates, shared_col_updates, shared_data_updates, compiled_updates, chunk):
    """
    Process a chunk of updates in parallel and accumulate them in shared memory lists.
    """
    for original_idx in chunk:
        update_coo = compiled_updates[original_idx]
        shared_row_updates.extend([original_idx] * len(update_coo.data))
        shared_col_updates.extend(update_coo.col.tolist())
        shared_data_updates.extend(update_coo.data.tolist())

def compile_and_apply_updates_parallel(KNN_main, updates_dict, n_jobs=4):
    """
    Compile updates from updates_dict and apply them to the KNN_main CSR matrix in parallel using shared memory.
    """
    # Compile the latest updates for each unique index
    compiled_updates = {}
    for epoch, (indices, KNN_hop) in sorted(updates_dict.items(), reverse=True):
        for idx, original_idx in enumerate(indices):
            if original_idx not in compiled_updates:
                compiled_updates[original_idx] = KNN_hop[idx, :].tocoo()

    # Initialize shared memory lists
    manager = Manager()
    shared_row_updates = manager.list()
    shared_col_updates = manager.list()
    shared_data_updates = manager.list()

    # Prepare chunks of updates for parallel processing
    chunks = np.array_split(list(compiled_updates.keys()), n_jobs)

    # Process updates in parallel
    pool = Pool(processes=n_jobs)
    func = partial(process_update_chunk, shared_row_updates, shared_col_updates, shared_data_updates, compiled_updates)
    pool.map(func, chunks)
    pool.close()
    pool.join()

    # Create a COO matrix from the compiled updates in shared memory
    updates_coo = coo_matrix((list(shared_data_updates), (list(shared_row_updates), list(shared_col_updates))), shape=KNN_main.shape)

    # Convert COO to CSR and apply updates
    KNN_main += updates_coo.tocsr()

    return KNN_main


In [4]:
knn = compile_and_apply_updates_parallel(KNN_main, updates_dict)

NameError: name 'KNN_main' is not defined

In [80]:
KNN_tmp = knn[adata.obs.index.isin(adata_samp.obs.index)]
KNN_tmp.data = np.where(KNN_tmp.data > 0, 1, 0)

indices = np.where(KNN_tmp.sum(axis=0) <= model_params['alpha'])[1]
len(indices)

2144